# 6.2 Setting up a custom DraCor infrastructure
This notebook demonstrates how to quickly setup a containerized local DraCor infrastructure. We will copy a corpus from the current DraCor live production system at https://dracor.org.

In the following cell we import the *stable dracor client* and setup an empty DraCor system. The components of the infrastructure (eXist database with API, metrics service, triple store) are started as Docker containers inside the Docker container we are working in right now:

In [ ]:
%%bash

# Stop and remove all Docker containers to avoid conflicts 
# especially regarding ports in the next section
# This cell does not show up in the final rendering of the HTML
# If you have set up custom DraCor Docker containers you should not run 
# this command. It will remove your work!

#docker stop $(docker ps -a -q)
#docker rm $(docker ps -a -q)

In [ ]:
from stabledracor.client import StableDraCor
dracor = StableDraCor()
dracor.run()

After the cell above has been executed (it should return `True`) the empty system can be accessed at http://localhost:8088.

We then copy the *Tatar Drama Corpus* (TatDraCor) from the live production instance at https://dracor.org to the locally running system in the Docker container: 

In [ ]:
dracor.copy_corpus(source_corpusname="tat")

Check http://localhost:8088/tat to view the newly added corpus. Copying a live corpus is a quick way to get something up and running locally. To make it possible to fully track the composition of a corpus and make research with DraCor repeatable, it is recommended to load corpora from GitHub instead. 

In the following cell we load a corpus from a GitHub repository of the [dracor-org GitHub organization](https://github.com/dracor-org). We load the [Polish Drama Corpus](https://github.com/dracor-org/poldracor). The name of the repository is`poldracor`:

In [ ]:
dracor.add_corpus_from_repo(repository_name="poldracor")

We can also create custom corpora and add plays from several corpora to them. First, we prepare the metadata of the corpus:

In [ ]:
hamlet_corpus_metadata = {
    "name" : "ham",
    "title" : "Hamlet",
    "description" : "A corpus containing Shakespeare's 'Hamlet' and some translations"
}

dracor.add_corpus(corpus_metadata=hamlet_corpus_metadata)

We have now create an empty corpus `ham`. We will add two plays from two DraCor corpora. 

In [ ]:
dracor.add_play_version_to_corpus(repository_name="shakedracor",
                                  filename="hamlet",
                                  corpusname="ham")

The second play we add to the corpus is the German translation by August Wilhelm Schlegel from the German Shakespeare Corpus (`gershdracor`).

In [ ]:
dracor.add_play_version_to_corpus(repository_name="gershdracor",
                                  filename="hamlet-prinz-von-daenemark",
                                  corpusname="ham")

In the following, more complex example we will setup a corpus of a single author. We will use the production instance of dracor at dracor.org to extract the identifiers `playname` of the plays by a certain author and then add these plays from the source repository to the local DraCor instance.

In [ ]:
from stabledracor.client import api_get
gerdracor_plays_from_production = api_get(api_base_url="https://dracor.org/api/v1/", 
                                          corpusname="ger")["plays"]

An easy way to filter is to loop over all the plays and add the playnames to a list if some criteria are met. In this case we look for the `shortname` `"Schiller"` in the `authors` field. 

In [ ]:
schiller_plays = []
for play in gerdracor_plays_from_production:
    if play["authors"][0]["shortname"] == "Schiller":
        schiller_plays.append(play["name"])

In [ ]:
schiller_plays

In [ ]:
schiller_corpus_metadata = {
    "name" : "schiller",
    "title" : "Schiller",
    "description" : "A corpus containing Friedrich Schiller's plays from the GerDraCor corpus"
}

dracor.add_corpus(corpus_metadata=schiller_corpus_metadata)

for playname in schiller_plays:
    dracor.add_play_version_to_corpus(repository_name="gerdracor", 
                                      filename=playname,
                                      corpusname="schiller")

## Task:
Set up your own local DraCor infrastructure. 

Running the following cell will stop and remove all docker images and allow you to start from scratch. 

In [ ]:
%%bash

# Stop and remove all Docker containers to avoid conflicts 
# especially regarding ports in the next section
# This cell does not show up in the final rendering of the HTML
# If you have set up custom DraCor Docker containers you should not run 
# this command. It will remove your work!

docker stop $(docker ps -a -q)
docker rm $(docker ps -a -q)